In [ ]:
import pyopencl as cl
from pyopencl.reduction import ReductionKernel
from pyopencl import array
import numpy as np
import time
import datetime
import os.path

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[1]
context = cl.Context([device])

In [ ]:
tic = time.clock()

scale = 4
corrosion_depth = 5
dt = 0.01

# Define time paramters
T_h = 0.5     # heating time
T = 0.5    # total simulation time
theta = 1
tol = 1E-6

# Define physical parameters, length scale is mm
T_init = 0
P = 10e9
omega = 2.
vert_scale = np.array([-15, -15, 0, 1./scale, 1./scale, 1./scale]).astype(np.float32)

# Define material parameters
rho_0 = 0.0076; rho_1 = 0.003;
C_0 = 4.9e8; C_1 = 5.5e8;
k_0 = 4.3e7; k_1 = 4e6;
mat_coefs = np.array([rho_0*C_0, k_0, rho_1*C_1, k_1]).astype(np.float32)

# Other FE arrays
C = np.array([30*scale, 30*scale, 10*scale]).astype(np.uint32)
nCells = C[0]*C[1]*C[2]*6
nVertices = (C[0]+1)*(C[1]+1)*(C[2]+1)
DoFMapLocal = np.array([0, 1, 3, 7,
                       0, 1, 5, 7,
                       0, 4, 5, 7,
                       0, 2, 3, 7,
                       0, 4, 6, 7,
                       0, 2, 6, 7]).astype(np.uint32)
DoFMapLocal2 = np.array([1, 33,
                   1, 65,
                   64, 65,
                   32, 33,
                   64, 96,
                   32, 96]).astype(np.uint32)
zMapLocal = np.array([0, 0,
                   0, vert_scale[5],
                   vert_scale[5], vert_scale[5],
                   0, 0,
                   vert_scale[5], vert_scale[5],
                   0, vert_scale[5]]).astype(np.float32)
with np.load('Scale_{}/meshData.npz'.format(scale)) as data:
    Mdata = data['Mdata']
    Kdata = data['Kdata']

M_FG = np.zeros(4, cl.array.vec.float4)
K_FG = np.zeros(4, cl.array.vec.float4)
for i in range(4):
    M_FG[i] = (Mdata[4*i + 0], Mdata[4*i + 1], Mdata[4*i + 2], Mdata[4*i + 3])
    K_FG[i] = (Kdata[4*i + 0], Kdata[4*i + 1], Kdata[4*i + 2], Kdata[4*i + 3])

Ndt = np.zeros(nVertices).astype(np.float32)
Ndt[0:(C[0]+1)*(C[1]+1)-1] = 1
# Ndt = N_host
# Ndt[(C[0]+1)*(C[1]+1):nVertices] = 0
# Ndt = N*dt
# Ndt = (Ndt>0).astype(np.float32)

x = np.ones(nVertices).astype(np.float32)
u_0 = T_init*np.ones_like(x)
Ax_split = np.zeros(nVertices*4).astype(np.float32)
P_split = np.zeros(nVertices*4).astype(np.float32)
corr_bounds = np.array([corrosion_depth]).astype(np.float32)
theta = np.float32(theta)
dt = np.float32(dt)
delta = np.array([1,]).astype(np.float32)
delta_new = np.empty_like(delta)
nC_with_padding = (C[0]+1)*(C[1]+1)*C[2]*6

toc = time.clock()
print "load data                {}".format(toc - tic)

In [ ]:
compiler_options = '-cl-mad-enable -cl-fast-relaxed-math'
# compiler_options = ''
Jacobi_A_prg = cl.Program(context, """
    __kernel void J_A(__constant float4 *M_FG, __constant float4 *K_FG,
    __constant uint *DoFMapLocal2,
    __constant float *zMapLocal,
    __constant uint *C,
    __constant float *vertex_loc_scale,
    __constant float *corr_bounds,
    __constant float *material_coefs,
    float theta, float dt,
    __local float *P_split_local,
    __global float *P_split)
     {
        uint gid = get_global_id(0);
        uint wid = get_group_id(0);
        uint lid = get_local_id(0);
        uint numgroups = get_num_groups(0);
        
        uint global_cell6id = (gid)/6 - wid;
        uint local_cell6id = lid/6;
        uint global_read_row = lid/32;
        uint private_cell6id = lid % 6;
        
        uint Cx = C[0];
        uint Cy = C[1];
        uint Cz = C[2];
        uint CxV = Cx+1;
        uint CyV = Cy+1;
        uint CzV = Cz+1;
        
        uint wg_base_id = wid*30;
        uint wgZ = wg_base_id/(CxV*CyV);
        uint wgY = (wg_base_id - wgZ*(CxV*CyV))/CxV;
        uint wgX = wg_base_id - wgZ*CxV*CyV - wgY*CxV;
        uint wg2g_offset = wgX + CxV*wgY + CxV*CyV*wgZ;

        uint global_read_ind;
        bool global_read = true;
        switch (global_read_row){
            case 0:
                global_read_ind = wg2g_offset + lid;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 1:
                global_read_ind = wg2g_offset + (lid-32) + CxV;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 2:
                global_read_ind = wg2g_offset + (lid-64) + CxV*CyV;
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            case 3:
                global_read_ind = wg2g_offset + (lid-96) + CxV*(CyV+1);
                for(int P_split_reset = 0; P_split_reset<12; P_split_reset++){
                P_split_local[lid*12 + P_split_reset] = 0;}
                break;
            default:
                global_read = false;
        }
        
        uint nCells = CxV*CyV*Cz*6;
        uint i, j, k, l;
        i = local_cell6id;
        j = local_cell6id + DoFMapLocal2[2*private_cell6id];
        k = local_cell6id + DoFMapLocal2[2*private_cell6id + 1];
        l = local_cell6id + 97;

        uint c6Z = global_cell6id/(CxV*CyV);
        uint c6Y = (global_cell6id - c6Z*(CxV*CyV))/CxV;
        uint c6X = global_cell6id - c6Z*CxV*CyV - c6Y*CxV;
        
        float vX = (c6X * vertex_loc_scale[3]) + vertex_loc_scale[0];
        float vY = (c6Y * vertex_loc_scale[4]) + vertex_loc_scale[1];
        float vZ = (c6Z * vertex_loc_scale[5]) + vertex_loc_scale[2];
        
        float M_coef = 0;
        float K_coef = 0;
        if(vZ>corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + vertex_loc_scale[5] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id + 1] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        
        M_coef = M_coef/4.f;
        K_coef = K_coef/4.f;
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        bool cell_in_bounds = (gid - 6*wid < nCells);
        float P_split_local_contribution;
        if(cell_in_bounds && (c6X < CxV-1) && (c6Y < CyV-1)){
            P_split_local_contribution = M_coef*M_FG[0].s0 + theta*dt*K_coef*K_FG[0].s0;
            P_split_local[i*12 + private_cell6id] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[1].s1 + theta*dt*K_coef*K_FG[1].s1;
            P_split_local[j*12 + private_cell6id + 6] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[2].s2 + theta*dt*K_coef*K_FG[2].s2;
            P_split_local[k*12 + private_cell6id] = P_split_local_contribution;
            P_split_local_contribution = M_coef*M_FG[3].s3 + theta*dt*K_coef*K_FG[3].s3;
            P_split_local[l*12 + private_cell6id + 6] = P_split_local_contribution;}
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        uint global_write_ind = global_read_ind*4 + global_read_row;
        bool vertex_in_bounds = (global_write_ind < CxV*CyV*CzV*4);
        float P_split_contribution = 0;
        if(global_read && vertex_in_bounds &&
        ((lid % 32 < 31) || (wid == numgroups-1)) && ((lid % 32 > 0) || (wid == 0))){
        for(uint vertex_neigbor = 0; vertex_neigbor < 12; vertex_neigbor++){
            P_split_contribution += P_split_local[lid*12 + vertex_neigbor];
        }
            P_split[global_write_ind] = P_split_contribution;
        }
    }
    """).build(options=compiler_options)

FGEbEMVP_A_prg = cl.Program(context, """
    __kernel void FGEbEMVP_A(__constant float4 *M_FG, __constant float4 *K_FG, 
    __global float *x,
    __constant uint *DoFMapLocal2,
    __constant float *zMapLocal,
    __constant uint *C,
    __constant float *vertex_loc_scale,
    __constant float *corr_bounds,
    __constant float *material_coefs,
    float theta, float dt,
    __local float *x_local,
    __local float *Ax_split_local,
    __global float *Ax_split)
    {
        uint gid = get_global_id(0);
        uint wid = get_group_id(0);
        uint lid = get_local_id(0);
        uint numgroups = get_num_groups(0);
        
        uint global_cell6id = (gid)/6 - wid;
        uint local_cell6id = lid/6;
        uint global_read_row = lid/32;
        uint private_cell6id = lid % 6;
        
        uint Cx = C[0];
        uint Cy = C[1];
        uint Cz = C[2];
        uint CxV = Cx+1;
        uint CyV = Cy+1;
        uint CzV = Cz+1;
        
        uint wg_base_id = wid*30;
        uint wgZ = wg_base_id/(CxV*CyV);
        uint wgY = (wg_base_id - wgZ*(CxV*CyV))/CxV;
        uint wgX = wg_base_id - wgZ*CxV*CyV - wgY*CxV;
        uint wg2g_offset = wgX + CxV*wgY + CxV*CyV*wgZ;

        uint global_read_ind;
        bool global_read = true;
        switch (global_read_row){
            case 0:
                global_read_ind = wg2g_offset + lid;
                break;
            case 1:
                global_read_ind = wg2g_offset + (lid-32) + CxV;
                break;
            case 2:
                global_read_ind = wg2g_offset + (lid-64) + CxV*CyV;
                break;
            case 3:
                global_read_ind = wg2g_offset + (lid-96) + CxV*(CyV+1);
                break;
            default:
                global_read = false;
        }
        
        if(global_read){
            x_local[lid] = x[global_read_ind];
        }
        
         switch (global_read_row){
            case 0:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 1:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 2:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
            case 3:
                for(int Ax_split_reset = 0; Ax_split_reset<12; Ax_split_reset++){
                Ax_split_local[lid*12 + Ax_split_reset] = 0;}
                break;
        }
        
        uint nCells = CxV*CyV*Cz*6;
        uint i, j, k, l;
        i = local_cell6id;
        j = local_cell6id + DoFMapLocal2[2*private_cell6id];
        k = local_cell6id + DoFMapLocal2[2*private_cell6id + 1];
        l = local_cell6id + 97;

        uint c6Z = global_cell6id/(CxV*CyV);
        uint c6Y = (global_cell6id - c6Z*(CxV*CyV))/CxV;
        uint c6X = global_cell6id - c6Z*CxV*CyV - c6Y*CxV;
        
        float vX = (c6X * vertex_loc_scale[3]) + vertex_loc_scale[0];
        float vY = (c6Y * vertex_loc_scale[4]) + vertex_loc_scale[1];
        float vZ = (c6Z * vertex_loc_scale[5]) + vertex_loc_scale[2];
        
        float M_coef = 0;
        float K_coef = 0;
        if(vZ>corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + vertex_loc_scale[5] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        if(vZ + zMapLocal[2*private_cell6id + 1] >corr_bounds[0]){
        M_coef += material_coefs[2]; K_coef += material_coefs[3];}
        else{ M_coef += material_coefs[0]; K_coef += material_coefs[1]; }
        
        M_coef = M_coef/4.f;
        K_coef = K_coef/4.f;
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        float Mx, Kx;
        
        bool cell_in_bounds = (gid - 6*wid < nCells);
        if(cell_in_bounds && (c6X < CxV-1) && (c6Y < CyV-1)){
            float4 x_cell = (float4)(x_local[i], x_local[j], x_local[k], x_local[l]);
            Mx = dot((M_FG[0]), x_cell); Kx = dot(K_FG[0], x_cell);
            float Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[i*12 + private_cell6id] = Ax_split_local_contribution;
            Mx = dot((M_FG[1]), x_cell); Kx = dot(K_FG[1], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[j*12 + private_cell6id + 6] = Ax_split_local_contribution;
            Mx = dot((M_FG[2]), x_cell); Kx = dot(K_FG[2], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[k*12 + private_cell6id] = Ax_split_local_contribution;
            Mx = dot((M_FG[3]), x_cell); Kx = dot(K_FG[3], x_cell);
            Ax_split_local_contribution = M_coef*Mx + theta*dt*K_coef*Kx;
            Ax_split_local[l*12 + private_cell6id + 6] = Ax_split_local_contribution;}
            
        barrier(CLK_LOCAL_MEM_FENCE);
        
        uint global_write_ind = global_read_ind*4 + global_read_row;
        bool vertex_in_bounds = (global_write_ind < CxV*CyV*CzV*4);
        float Ax_split_contribution = 0;
        if(global_read && vertex_in_bounds &&
        ((lid % 32 < 31) || (wid == numgroups-1)) && ((lid % 32 > 0) || (wid == 0))){
        for(uint vertex_neigbor = 0; vertex_neigbor < 12; vertex_neigbor++){
            Ax_split_contribution += Ax_split_local[lid*12 + vertex_neigbor];
        }
            Ax_split[global_write_ind] = Ax_split_contribution;
        }
        
    }
    """).build(options=compiler_options) 

FGEbEMVP_B_prg = cl.Program(context, """
    __kernel void FGEbEMVP_B(__global float *Ax_split,
    __global float *Ax)
    {
        uint gid = get_global_id(0);
        uint vertex_offset = gid * 4;
        float Ax_contribution = 0;
        
        for(uint i = 0; i < 4; i++){
            Ax_contribution += Ax_split[vertex_offset + i];
        }

        Ax[gid] = Ax_contribution;     
    }
    """).build(options=compiler_options)

FGEbEMVP_C_prg = cl.Program(context, """
    __kernel void FGEbEMVP_C(__global float *Ax_split,
    __global float *b,
    float c,
    __global float *cAx_plus_b)
    {
        uint gid = get_global_id(0);
        uint vertex_offset = gid * 4;
        float Ax_contribution = b[gid];
        
        for(uint i = 0; i < 4; i++){
            Ax_contribution += c*Ax_split[vertex_offset + i];
        }

        cAx_plus_b[gid] = Ax_contribution;     
    }
    """).build(options=compiler_options)

VVP_A_prg = cl.Program(context, """
    __kernel void VVP_A(__global float *x,
    __global float *y,
    uint N,
    __local float *VVP_loc,
    __global float *r)
    {
      uint gid = get_global_id(0);
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N) VVP_loc[lid] = (x[gid] * y[gid]);
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0) r[wid] = VVP_loc[lid];
    }
    """).build(options=compiler_options)

VVP_C_prg = cl.Program(context, """
    __kernel void VVP_C(__global float *r,
    __global float *delta,
    uint N,
    __local float *VVP_loc,
    __global float *alpha,
    __global float *neg_alpha)
    {
      uint gid = get_global_id(0);
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N) VVP_loc[lid] = r[gid];
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0){
          float alpha_temp = delta[0]/VVP_loc[lid];
          alpha[0] = alpha_temp;
          neg_alpha[0] = -alpha_temp;
      }
    }
    """).build(options=compiler_options)

VVP_reduce_prg = cl.Program(context, """
    __kernel void VVP_reduce(__global float *r1,
    uint N,
    __local float *VVP_loc,
    __global float *r2)
    {
      uint gid = get_global_id(0);
      uint wid = get_group_id(0);
      uint lid = get_local_id(0);
      uint gs = get_local_size(0);
      if(gid < N) VVP_loc[lid] = r1[gid];
      else VVP_loc[lid] = 0;
      barrier(CLK_LOCAL_MEM_FENCE);
      for(uint s = gs/2; s > 0; s >>= 1) {
        if(lid < s) {
          VVP_loc[lid] += VVP_loc[lid+s];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
      }
      if(lid == 0) r2[wid] = VVP_loc[lid];
    }
    """).build(options=compiler_options)

VAVSP_prg = cl.Program(context, """
    __kernel void VAVSP(__global float *x,
    __global float *y,
    __global float *a,
    __global float *x_plus_ay)
    {
        int gid = get_global_id(0);
        float x_temp;
        float y_temp;
        float c_temp;
        x_temp = x[gid];
        y_temp = y[gid];
        
        c_temp = x_temp + a[0]*y_temp;
        x_plus_ay[gid] = c_temp;     
    }
    """).build(options=compiler_options)

DIMVP_prg = cl.Program(context, """
    __kernel void DIMVP(__global float *P,
    __global float *x,
    __global float *Pinvx)
    {
        int gid = get_global_id(0);
        Pinvx[gid] = x[gid]/P[gid];     
    }
    """).build(options=compiler_options)

Beta_update_prg = cl.Program(context, """
    __kernel void Beta_update(__global float *delta_new,
    __global float *delta_old,
    __global float *beta)
    {
        beta[0] = delta_new[0]/delta_old[0];
        barrier(CLK_LOCAL_MEM_FENCE);
        delta_old[0] = delta_new[0];
    }
    """).build(options=compiler_options)

u0_update_prg = cl.Program(context, """
    __kernel void u0_update(__global float *u0,
    __global float *u_new)
    {
        int gid = get_global_id(0);
        u0[gid] = 2*u_new[gid] - u0[gid];
    }
    """).build(options=compiler_options)

In [ ]:
queue  = cl.CommandQueue(context)
mf = cl.mem_flags
map_flags = cl.map_flags
MVP_wg_size = np.uint32(6*31)
MVP_global_size = np.uint32(np.ceil((nC_with_padding/(MVP_wg_size-6.))) * MVP_wg_size)
max_wg_size = context.get_info(cl.context_info.DEVICES)[0].max_work_group_size

HOST_TO_DEVICE_COPY = (mf.READ_ONLY  | mf.HOST_WRITE_ONLY | mf.COPY_HOST_PTR)
HOST_TO_DEVICE_USE  = (mf.READ_ONLY  | mf.HOST_WRITE_ONLY | mf.USE_HOST_PTR)
HOST_READ_WRITE     = (mf.READ_WRITE | mf.COPY_HOST_PTR)
PINNED              = (mf.READ_WRITE | mf.USE_HOST_PTR)
DEVICE_READ_WRITE   = (mf.READ_WRITE | mf.HOST_NO_ACCESS)

M_FG_buf         = cl.Buffer(context, HOST_TO_DEVICE_USE, hostbuf = M_FG)
K_FG_buf         = cl.Buffer(context, HOST_TO_DEVICE_USE, hostbuf = K_FG)
DoFMapLocal2_buf = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = DoFMapLocal2)
zMapLocal_buf    = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = zMapLocal)
C_buf            = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = C)
vert_scale_buf   = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = vert_scale)
corr_bounds_buf  = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = corr_bounds)
mat_coefs_buf    = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = mat_coefs)

Ax_split_buf     = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = np.zeros_like(Ax_split))
P_split_buf      = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = np.zeros_like(P_split))
P_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)

u0_buf           = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = u_0)
Ndt_buf          = cl.Buffer(context, HOST_TO_DEVICE_COPY, hostbuf = Ndt)
x_buf            = cl.Buffer(context, HOST_READ_WRITE, hostbuf = np.zeros_like(x))
b_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)

r_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)
d_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)
q_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)
s_buf            = cl.Buffer(context, DEVICE_READ_WRITE, x.nbytes)

delta_buf        = cl.Buffer(context, PINNED, hostbuf = delta)
alpha_buf        = cl.Buffer(context, DEVICE_READ_WRITE, delta.nbytes)
neg_alpha_buf    = cl.Buffer(context, DEVICE_READ_WRITE, delta.nbytes)
delta_new_buf    = cl.Buffer(context, DEVICE_READ_WRITE, delta.nbytes)
beta_buf         = cl.Buffer(context, DEVICE_READ_WRITE, delta.nbytes)

x_local_buf        = cl.LocalMemory(4 * 32*4)
Ax_split_local_buf = cl.LocalMemory(4 * 32*4*12)
VVP_loc_buf        = cl.LocalMemory(4 * max_wg_size)

knl_PA = Jacobi_A_prg.J_A
knl_PA.set_args(M_FG_buf, K_FG_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf,
                vert_scale_buf, corr_bounds_buf, mat_coefs_buf, (np.float32(1)*theta), 
                dt, Ax_split_local_buf, P_split_buf)

knl_PB = FGEbEMVP_B_prg.FGEbEMVP_B
knl_PB.set_args(P_split_buf, P_buf)

knl_RHS_A = FGEbEMVP_A_prg.FGEbEMVP_A
knl_RHS_A.set_args(M_FG_buf, K_FG_buf, u0_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf,
                vert_scale_buf, corr_bounds_buf, mat_coefs_buf, (np.float32(-(1-theta))), 
                dt, x_local_buf, Ax_split_local_buf, Ax_split_buf)
knl_RHS_B = FGEbEMVP_C_prg.FGEbEMVP_C
knl_RHS_B.set_args(Ax_split_buf, Ndt_buf, np.float32(1), b_buf)

knl_1A = FGEbEMVP_A_prg.FGEbEMVP_A
knl_1A.set_args(M_FG_buf, K_FG_buf, x_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf,
                vert_scale_buf, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf, Ax_split_local_buf, Ax_split_buf)

knl_1B = FGEbEMVP_C_prg.FGEbEMVP_C
knl_1B.set_args(Ax_split_buf, b_buf, np.float32(-1), r_buf)

knl_2 = DIMVP_prg.DIMVP
knl_2.set_args(P_buf, r_buf, d_buf)

knl_4A = FGEbEMVP_A_prg.FGEbEMVP_A
knl_4A.set_args(M_FG_buf, K_FG_buf, d_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf,
                vert_scale_buf, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf, Ax_split_local_buf, Ax_split_buf)
knl_4B = FGEbEMVP_B_prg.FGEbEMVP_B
knl_4B.set_args(Ax_split_buf, q_buf)

knl_6 = VAVSP_prg.VAVSP
knl_6.set_args(x_buf, d_buf, alpha_buf, x_buf)

knl_7A = FGEbEMVP_A_prg.FGEbEMVP_A
knl_7A.set_args(M_FG_buf, K_FG_buf, x_buf, DoFMapLocal2_buf, zMapLocal_buf, C_buf,
                vert_scale_buf, corr_bounds_buf, mat_coefs_buf, (np.float32(theta)), 
                dt, x_local_buf, Ax_split_local_buf, Ax_split_buf)
knl_7B = FGEbEMVP_C_prg.FGEbEMVP_C
knl_7B.set_args(Ax_split_buf, b_buf, np.float32(-1), r_buf)

knl_7 = VAVSP_prg.VAVSP
knl_7.set_args(r_buf, q_buf, neg_alpha_buf, r_buf)

knl_8 = DIMVP_prg.DIMVP
knl_8.set_args(P_buf, r_buf, s_buf)

knl_10 = Beta_update_prg.Beta_update
knl_10.set_args(delta_new_buf, delta_buf, beta_buf)

knl_11 = VAVSP_prg.VAVSP
knl_11.set_args(s_buf, d_buf, beta_buf, d_buf)

knl_u0 = u0_update_prg.u0_update
knl_u0.set_args(u0_buf, x_buf)

if (nVertices <= max_wg_size * max_wg_size):
    reduction_steps = 1
    r1_size = np.uint32(np.ceil(nVertices/max_wg_size))
    global_red_size = r1_size * max_wg_size
    r1_buf = cl.Buffer(context, DEVICE_READ_WRITE, r1_size*4)
    knl_3A = VVP_A_prg.VVP_A
    knl_3A.set_args(r_buf, d_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_3B = VVP_reduce_prg.VVP_reduce
    knl_3B.set_args(r1_buf, r1_size, VVP_loc_buf, delta_buf)
    knl_5A = VVP_A_prg.VVP_A
    knl_5A.set_args(d_buf, q_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_5B = VVP_C_prg.VVP_C
    knl_5B.set_args(r1_buf, delta_buf, r1_size, VVP_loc_buf, alpha_buf, neg_alpha_buf)
    knl_9A = VVP_A_prg.VVP_A
    knl_9A.set_args(r_buf, s_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_9B = VVP_reduce_prg.VVP_reduce
    knl_9B.set_args(r1_buf, r1_size, VVP_loc_buf, delta_new_buf)
elif(nVertices <= max_wg_size * max_wg_size * max_wg_size):
    reduction_steps = 2
    r1_size = np.uint32(np.ceil(nVertices/max_wg_size))
    global_red1_size = r1_size * max_wg_size
    r2_size = np.uint32(np.ceil(r1_size/max_wg_size))
    global_red2_size = r2_size * max_wg_size
    r1_buf = cl.Buffer(context, DEVICE_READ_WRITE, r1_size*4)
    r2_buf = cl.Buffer(context, DEVICE_READ_WRITE, r2_size*4)
    knl_3A = VVP_A_prg.VVP_A
    knl_3A.set_args(r_buf, d_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_3B = VVP_reduce_prg.VVP_reduce
    knl_3B.set_args(r1_buf, r1_size, VVP_loc_buf, r2_buf)
    knl_3C = VVP_reduce_prg.VVP_reduce
    knl_3C.set_args(r2_buf, r2_size, VVP_loc_buf, delta_buf)
    knl_5A = VVP_A_prg.VVP_A
    knl_5A.set_args(d_buf, q_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_5B = VVP_reduce_prg.VVP_reduce
    knl_5B.set_args(r1_buf, r1_size, VVP_loc_buf, r2_buf)
    knl_5C = VVP_C_prg.VVP_C
    knl_5C.set_args(r2_buf, delta_buf, r2_size, VVP_loc_buf, alpha_buf, neg_alpha_buf)
    knl_9A = VVP_A_prg.VVP_A
    knl_9A.set_args(r_buf, s_buf, nVertices, VVP_loc_buf, r1_buf)
    knl_9B = VVP_reduce_prg.VVP_reduce
    knl_9B.set_args(r1_buf, r1_size, VVP_loc_buf, r2_buf)
    knl_9C = VVP_reduce_prg.VVP_reduce
    knl_9C.set_args(r2_buf, r2_size, VVP_loc_buf, delta_new_buf)


In [ ]:
%%timeit -t -o -n 1 -r 5
# Conjugate gradient to solve A*x = b (or for us, A*u_i = L*u_{i-1} + N)

# Initialization
# tic_Total = time.clock()

tol = 1e-6
i_max = 200
t = 0
total_iterations = 0 

# Profiling variables
        
# Reset u0
u0_init_event = cl.enqueue_copy(queue, u0_buf, np.zeros_like(x))
x_init_event = cl.enqueue_copy(queue, x_buf, np.zeros_like(u_0), wait_for = [u0_init_event])
x_init_event.wait()

# Apply corrosion profile for coefficient scaling of A and L, and gather data for preconditioner P
event_PA = cl.enqueue_nd_range_kernel(queue, knl_PA, (MVP_global_size,), (MVP_wg_size,))
# Consolidate P
event_PB = cl.enqueue_nd_range_kernel(queue, knl_PB, (nVertices,), None, wait_for = [event_PA])

while(t <= T_h):
    # Compute RHS vector for PCG step i, b = L*u_{i-1} + N
#     tic_RHS = time.clock()
    event_RHS_A = cl.enqueue_nd_range_kernel(queue, knl_RHS_A, (MVP_global_size,), (MVP_wg_size,))
    # Consolidate b
    event_RHS_B = cl.enqueue_nd_range_kernel(queue, knl_RHS_B, (nVertices,), None, wait_for = [event_RHS_A])
#     elapsed_RHS = time.clock() - tic_RHS

    # Preconditioned Conjugate Gradient to solve A*x = b
    tic_PCG = time.clock()
    i = 0

    # 1A) Ax = A*x_init      (MVP)
    event_1A = cl.enqueue_nd_range_kernel(queue, knl_1A, (MVP_global_size,), (MVP_wg_size,))

    # 1B) r = b - Ax      (VAVSP)
    event_1B = cl.enqueue_nd_range_kernel(queue, knl_1B, (nVertices,), None, wait_for = [event_1A])

    # 2) d = Pinv * r      (DIMVP)
    event_2 = cl.enqueue_nd_range_kernel(queue, knl_2, (nVertices,), None, wait_for = [event_1B, event_PB])

    # 3) delta_new = r' * d      (VVP)
    if (reduction_steps == 1):
        event_3A = cl.enqueue_nd_range_kernel(queue, knl_3A, (global_red_size,), (max_wg_size,), wait_for = [event_2])
        event_3C = cl.enqueue_nd_range_kernel(queue, knl_3B, (max_wg_size,), (max_wg_size,), wait_for = [event_3A])
    elif (reduction_steps == 2):
        event_3A = cl.enqueue_nd_range_kernel(queue, knl_3A, (global_red1_size,), (max_wg_size,), wait_for = [event_2])
        event_3B = cl.enqueue_nd_range_kernel(queue, knl_3B, (global_red2_size,), (max_wg_size,), wait_for = [event_3A])
        event_3C = cl.enqueue_nd_range_kernel(queue, knl_3C, (max_wg_size,), (max_wg_size,), wait_for = [event_3B])
        
    # Map delta to host
    event_mapDelta = cl.enqueue_map_buffer(queue, delta_buf, map_flags.READ, 0, (1), np.float32, wait_for = [event_3C])  
    event_mapDelta[1].wait()
    
    # Inner loop:
    while (i < i_max) & (np.sqrt(delta) > tol):
        # 4) q = A * d      (MVP)
        event_4A = cl.enqueue_nd_range_kernel(queue, knl_4A, (MVP_global_size,), (MVP_wg_size,))
        event_4B = cl.enqueue_nd_range_kernel(queue, knl_4B, (nVertices,), None, wait_for = [event_4A])

        # 5) alpha = delta_new/(d' * q)      (VVP)
        if (reduction_steps == 1):
            event_5A = cl.enqueue_nd_range_kernel(queue, knl_5A, (global_red_size,), (max_wg_size,), wait_for = [event_4B])
            event_5C = cl.enqueue_nd_range_kernel(queue, knl_5B, (max_wg_size,), (max_wg_size,), wait_for = [event_5A])
        elif (reduction_steps == 2):
            event_5A = cl.enqueue_nd_range_kernel(queue, knl_5A, (global_red1_size,), (max_wg_size,), wait_for = [event_4B])
            event_5B = cl.enqueue_nd_range_kernel(queue, knl_5B, (global_red2_size,), (max_wg_size,), wait_for = [event_5A])    
            event_5C = cl.enqueue_nd_range_kernel(queue, knl_5C, (max_wg_size,), (max_wg_size,), wait_for = [event_5B])

        # 6) x = x + alpha * d      (VAVSP)
        event_6 = cl.enqueue_nd_range_kernel(queue, knl_6, (nVertices,), None, wait_for = [event_5C])
            
        # Optional correction step
        if ((i % 50) == 1):
        # 7) r = b - A*x      (MVP, VAVSP)
            event_7A = cl.enqueue_nd_range_kernel(queue, knl_7A, (MVP_global_size,), (MVP_wg_size,), wait_for = [event_6])
            event_7B = cl.enqueue_nd_range_kernel(queue, knl_7B, (nVertices,), None, wait_for = [event_7A])
        else:
        # 7) r = r - alpha*q      (VAVSP)
            event_7B = cl.enqueue_nd_range_kernel(queue, knl_7, (nVertices,), None, wait_for = [event_6])

        # 8) s = Pinv * r      (DIMVP)
        event_8 = cl.enqueue_nd_range_kernel(queue, knl_8, (nVertices,), None, wait_for = [event_7B])

        # 9) delta_new = r' * s       (VVP)
        if (reduction_steps == 1):
            event_9A = cl.enqueue_nd_range_kernel(queue, knl_9A, (global_red_size,), (max_wg_size,), wait_for = [event_8])
            event_9C = cl.enqueue_nd_range_kernel(queue, knl_9B, (max_wg_size,), (max_wg_size,), wait_for = [event_9A])
        elif (reduction_steps == 2):
            event_9A = cl.enqueue_nd_range_kernel(queue, knl_9A, (global_red1_size,), (max_wg_size,), wait_for = [event_8])
            event_9B = cl.enqueue_nd_range_kernel(queue, knl_9B, (global_red2_size,), (max_wg_size,), wait_for = [event_9A])
            event_9C = cl.enqueue_nd_range_kernel(queue, knl_9C, (max_wg_size,), (max_wg_size,), wait_for = [event_9B])

        # 10) beta = delta_new/delta_old
        event_10 = cl.enqueue_nd_range_kernel(queue, knl_10, (1,), (1,), wait_for = [event_9C])

        # 11) d = s + beta * d      (VAVSP)
        event_11 = cl.enqueue_nd_range_kernel(queue, knl_11, (nVertices,), None, wait_for = [event_10])

        # Map delta to host, sometimes
        if ((i % 1) == 0):
            event_mapDelta = cl.enqueue_map_buffer(queue, delta_buf, map_flags.READ, 0, (1), np.float32, wait_for = [event_11])
            
        i += 1
        event_11.wait()
        
    # Set u_{i-1} = u_i
    event_u0 = cl.enqueue_nd_range_kernel(queue, knl_u0, (nVertices,), None)
    
    total_iterations += i
    t += dt
    event_u0.wait()
        
print("Total iterations: %g" % (total_iterations))

In [ ]:
IPython_time = _
print dir(IPython_time)
print IPython_time.all_runs
print np.mean(IPython_time.all_runs)

In [ ]:
x_result = np.empty_like(x)
x_copy_event = cl.enqueue_copy(queue, x_result, x_buf)
x_copy_event.wait()

x_axis = np.linspace(0, nVertices, nVertices)
h = plt.plot(x_axis, x_result)
# h = plt.plot(x_axis[0:nVertices:(C[0]+1)*(C[1]+1)], x_result[0:nVertices:(C[0]+1)*(C[1]+1)])
print np.min(x_result)
print nVertices